
# Parsl Fest 2025: Parallel Notebook Testing in QMCPy with Parsl


Joshua Herman and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

**Requirements**:

* QMCPy: `pip install qmcpy==1.6.1.2`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
import qmcpy
import sys
import os

In [ ]:
!(cd .. && make booktests)


Booktests
cd test/booktests/ && \
	PYTHONWARNINGS="ignore::UserWarning,ignore::DeprecationWarning,ignore::FutureWarning,ignore::ImportWarning" \
	python -m coverage run --append --source=. -m unittest discover -s . -p "tb_*.py" && \
	sleep 5 && \
	rm -f *.eps *.jpg *.pdf *.part && \
	cd ../..
ssssss.ss.ss.sss.s..ss

## 2. Parsl

1. Install and Configure Parsl
2. Run the Notebook in Parallel with Parsl

### 2.1 Install and Configure Parsl

In [ ]:
import parsl as pl
from parsl.configs.htex_local import config

if not pl.config:
    pl.load(config)

### 2.2 Create a Parsl Test Runner

In [ ]:
# Ensure the path to the booktests directory is included
sys.path.append(os.path.join(os.getcwd(), '..', 'test', 'booktests'))

In [ ]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")

print("\n" + "="*50)

# Get help on specific function
print("Help for run_single_test:")
help(parsl_test_runner.execute_parallel_tests)

### 2.3 Run the Notebook in Parallel with Parsl

In [ ]:
!(cd .. && make booktests-parallel)

: 